In [8]:
from casadi import *
import numpy as np
import sympy as sp
import pickle

In [38]:
# importer = Importer('quad_casadi_dynamics.c', 'shell')
rhs = external('rhs', 'tmp_casadi_compiler.so')
state = MX.sym("x", 12)
param = MX.sym("p", 24)
# state_real = np.linspace(1, 10, 12)
# parameters_real = np.linspace(0.4, 4, 24)
rhs = rhs(state, param)
# rhs = Function("rhs",[state,param],[rhs])
# print(rhs(state_real, parameters_real))
# #%timeit -r 20 rhs(state_real, parameters_real)


In [82]:
dae = DaeBuilder('dog', '/home/slava/Desktop/Docker/dynamics/vdp_generated')
dae.disp(True)

f = dae.create('f', ['x', 'u'], ['ode'])

nx = 0, nz = 0, nq = 0, ny = 0, np = 0, nc = 0, nd = 0, nw = 0, nu = 1
Variables
  u = [u]


CasADi - 2023-06-23 18:21:15 WARNING("Unknown type for x") [.../casadi/core/dae_builder_internal.cpp:2576]
CasADi - 2023-06-23 18:21:15 WARNING("Unknown type for u") [.../casadi/core/dae_builder_internal.cpp:2576]
CasADi - 2023-06-23 18:21:15 WARNING("Unknown type for der_x") [.../casadi/core/dae_builder_internal.cpp:2576]


RuntimeError: Error in DaeBuilder::create for 'dog'  at .../casadi/core/dae_builder.cpp:737:
.../casadi/core/dae_builder_internal.cpp:2271: Fmu::init() failed: .../casadi/core/casadi_os.cpp:166: Assertion "handle!=nullptr" failed:
DllLibrary::init_handle: Cannot load shared library '/home/slava/Desktop/Docker/dynamics/vdp_generated/binaries/linux64/vdp.so': 
   (
    Searched directories: 1. casadipath from GlobalOptions
                          2. CASADIPATH env var
                          3. PATH env var (Windows)
                          4. LD_LIBRARY_PATH env var (Linux)
                          5. DYLD_LIBRARY_PATH env var (osx)
    A library may be 'not found' even if the file exists:
          * library is not compatible (different compiler/bitness)
          * the dependencies are not found
   )
  Tried '/home/slava/Desktop/Docker/venv/lib/python3.9/site-packages/casadi' :
    Error code: /home/slava/Desktop/Docker/venv/lib/python3.9/site-packages/casadi//home/slava/Deskto

In [69]:
print("!!!!!!!!!")
dae

!!!!!!!!!


DaeBuilder(nx = 0, nz = 0, nq = 0, ny = 0, np = 0, nc = 0, nd = 0, nw = 0, nu = 1)

In [21]:
opti = Opti()

x_dim = 12
u_dim = 12

foot_num = 4

N = 5
dt = 0.03
mu = 0.3

# with open("casadi_rhs.txt", "rb") as outf:
#     rhs = pickle.load(outf)


def euler_integrator(x0, u, p, dt):
    return x0 + dt * rhs(x0, vcat([p, u]))


x = opti.variable(x_dim * N)
u = opti.variable(u_dim * (N - 1))
x0 = opti.parameter(x_dim)
x_ref = opti.parameter(x_dim * N)
x_foots = opti.parameter(3 * foot_num * N)
contacts = opti.parameter(N, foot_num)

# friction cone
cone = np.array([[1, 0, -mu], [-1, 0, -mu], [0, 1, -mu], [0, -1, -mu]])
friction_constraints = kron(np.eye(4), cone)

u_ref = MX.zeros(u_dim * (N - 1))  #np.zeros(u_dim * (N - 1))
for i in range(N - 1):
    for leg in range(foot_num):
        u_ref[i * u_dim + leg * 3 + 2] = 13.74 * 9.81 / sum2(contacts[i, :])
# u_ref = [0] * u_dim * (N - 1)  #np.zeros(u_dim * (N - 1))
# u_ref[2::3] = [13.74 * 9.81 / sum2(contacts[i, :])
#                for i in range(N)]  # every z (3rd) component is non zero
# TO DO fix count of contacts in u_ref

dx = x - x_ref
du = u - u_ref

weights = [1., 1., 10., 0.1, 0.1, 0.1, 0.02, 0.02, 0.04, 0.01, 0.01, 0.002]

Q = np.diag(weights * N)
R = np.diag([2.5e-05] * u_dim * (N - 1))

weighted_sum = dx.T @ Q @ dx + du.T @ R @ du
opti.minimize(weighted_sum)

opti.subject_to(x[0:x_dim] == x0)
for n in range(N - 1):
    opti.subject_to(x[(n + 1) * x_dim:(n + 2) *
                      x_dim] == euler_integrator(x[n * x_dim:(n + 1) * x_dim],
                                                 u[n * u_dim:(n + 1) * u_dim],
                                                 x_foots[n * u_dim:(n + 1) *
                                                         u_dim],
                                                 dt=dt))

    contact_constraints = kron(diag(1 - contacts[n, :]), np.eye(3))
    opti.subject_to(contact_constraints @ u[n * u_dim:(n + 1) * u_dim] == 0)

    opti.subject_to(friction_constraints @ u[n * u_dim:(n + 1) * u_dim] <= 0)

opti_options = {'print_in': False, 'print_out': False, 'print_time': True}
solver_options = {'print_level': 0}

opti.solver('ipopt', opti_options, solver_options)
# mpc = opti.to_function("MPC", [x0, x_ref, x_foots, contacts], [u],
#                        ["x0", "x_ref", "x_foots", "contacts"], ["u_opt"])

mpc = opti.to_function("MPC", [x0, x_ref, x_foots, contacts], [u, x],
                       ["x0", "x_ref", "x_foots", "contacts"], ["u_opt", "x"])


RuntimeError: Error in Opti::to_function [OptiNode] at .../casadi/core/optistack.cpp:331:
Error in Opti::to_function [OptiNode] at .../casadi/core/optistack.cpp:331:
.../casadi/core/function_internal.cpp:145: Error calling IpoptInterface::init for 'solver':
Error in Function::factory for 'nlp' [MXFunction] at .../casadi/core/function.cpp:1747:
Failed to create nlp_grad:[x, p, lam:f, lam:g]->[f, g, grad:gamma:x, grad:gamma:p] with {"gamma": [f, g]}:
.../casadi/core/factory.hpp:584: Gradient generation failed:
Error in MX::gradient at .../casadi/core/generic_matrix.hpp:1466:
Error in MX::jtimes at .../casadi/core/generic_matrix.hpp:1454:
Error in MX::reverse at .../casadi/core/mx.cpp:1788:
Error in MXFunction::ad_reverse at .../casadi/core/mx_function.cpp:1101:
Error in MX::ad_reverse for node of type N6casadi4CallE at .../casadi/core/mx.cpp:2310:
Error in Call::ad_reverse for 'rhs' [MXFunction] at .../casadi/core/casadi_call.cpp:149:
Error in Function::reverse for 'rhs' [MXFunction] at 

# Tests

## Integrator test

In [ ]:
dt = 0.1
state_real = np.linspace(1, 10, 12).reshape(12, 1)
parameters_real = np.linspace(0.4, 4, 24).reshape(24, 1)
foot_pose_real = parameters_real[:12]
grf_real = parameters_real[12:]

true_diff = np.array([[5.90909091], [6.72727273], [7.54545455], [-17.87348989],
                      [-5.65715127], [28.9946412], [0.86829947], [0.91386621],
                      [-8.85056705], [-80.01278184], [96.99788936],
                      [-2.48111842]])
res = euler_integrator(state_real, grf_real, foot_pose_real,
                       dt).full() - (state_real + dt * true_diff)
if np.all(np.abs(res) < 1e-9):
    print("Euler seems good")


Euler seems good


## Contact constraints test

In [ ]:
contacts_real = 1 - SX([1, 0, 0, 1])
contact_fun = Function("krone", [], [kron(diag(contacts_real), np.eye(3))])
res = contact_fun()["o0"].full() @ grf_real
res = np.all(res[:3] == 0) and np.all(res[3:9] != 0) and np.all(res[9:] == 0)
if (res):
    print("contact constraints seems good")
else:
    print("constraints fail")


contact constraints seems good


## Friction cone constraints

In [ ]:
res_friction = friction_constraints @ grf_real <= 0
tmp = np.array([[1, 0, -mu], [-1, 0, -mu], [0, 1, -mu], [0, -1, -mu]])
res = np.all([
    res_friction[:4].full() == (tmp @ grf_real[:3] <= 0),
    res_friction[4:8].full() == (tmp @ grf_real[3:6] <= 0),
    res_friction[8:12].full() == (tmp @ grf_real[6:9] <= 0),
    res_friction[12:16].full() == (tmp @ grf_real[9:12] <= 0)
])
if (res):
    print("fricton constraints seems good")
else:
    print("friction fails")


fricton constraints seems good


## MPC Test

### Real data

In [ ]:
grf_positions_b = """[  0.1753,  0.08733,  -0.2813,  -0.1507,   0.1956,  -0.3318,   0.1618, -0.06079,  -0.3319,  -0.1972,   -0.145,  -0.2817]
[  0.1787,  0.09574,  -0.2847,   -0.154,   0.1911,  -0.3315,   0.1585, -0.06523,  -0.3316,  -0.1921,  -0.1373,  -0.2852]
[   0.183,   0.1049,  -0.2937,  -0.1564,   0.1873,  -0.3311,   0.1561,  -0.0691,  -0.3312,   -0.186,  -0.1289,   -0.294]
[  0.1865,   0.1131,  -0.3052,  -0.1588,    0.184,  -0.3305,   0.1537, -0.07238,  -0.3305,   -0.181,  -0.1213,  -0.3054]
[  0.1886,   0.1196,  -0.3167,  -0.1615,   0.1814,  -0.3299,    0.151, -0.07498,  -0.3299,  -0.1778,  -0.1153,  -0.3168]
[  0.1886,   0.1238,  -0.3258,  -0.1646,   0.1795,  -0.3296,   0.1479, -0.07683,  -0.3297,  -0.1769,  -0.1114,  -0.3258]
[  0.1859,    0.125,  -0.3299,  -0.1685,   0.1785,    -0.33,    0.144, -0.07781,  -0.3301,  -0.1793,  -0.1103,  -0.3299]
[  0.1811,    0.125,  -0.3314,  -0.1705,   0.1777,  -0.3277,   0.1425, -0.07927,  -0.3278,  -0.1841,  -0.1104,  -0.3314]
[  0.1762,   0.1251,  -0.3309,  -0.1678,   0.1754,   -0.318,   0.1468, -0.08318,  -0.3181,  -0.1891,  -0.1103,  -0.3309]
[  0.1711,   0.1252,  -0.3289,  -0.1613,    0.172,  -0.3039,   0.1557, -0.08912,  -0.3039,  -0.1942,  -0.1102,  -0.3289]
[  0.1656,   0.1253,  -0.3257,  -0.1522,   0.1677,  -0.2887,   0.1677, -0.09659,  -0.2887,  -0.1996,  -0.1101,  -0.3257]
[  0.1598,   0.1254,  -0.3218,  -0.1415,   0.1627,  -0.2755,   0.1817,  -0.1051,  -0.2755,  -0.2055,    -0.11,  -0.3218]
[  0.1534,   0.1253,  -0.3175,  -0.1304,   0.1574,  -0.2675,   0.1963,  -0.1142,  -0.2675,  -0.2119,    -0.11,  -0.3175]
[  0.1464,   0.1253,  -0.3131,  -0.1199,    0.152,  -0.2668,   0.2103,  -0.1234,  -0.2668,  -0.2188,  -0.1101,  -0.3131]
[   0.139,   0.1252,   -0.309,  -0.1108,   0.1468,   -0.272,   0.2227,  -0.1321,   -0.272,  -0.2262,  -0.1102,   -0.309]
[  0.1312,   0.1251,  -0.3054,  -0.1041,   0.1424,  -0.2804,   0.2324,  -0.1397,  -0.2804,  -0.2341,  -0.1102,  -0.3054]
[   0.123,   0.1252,  -0.3023,  -0.1007,   0.1389,  -0.2894,   0.2381,  -0.1456,  -0.2894,  -0.2422,  -0.1101,  -0.3023]
[  0.1145,   0.1255,  -0.2998,  -0.1015,   0.1369,  -0.2961,   0.2388,  -0.1493,  -0.2961,  -0.2508,  -0.1098,  -0.2998]
[  0.1056,   0.1261,  -0.2978,  -0.1076,   0.1366,  -0.2978,   0.2333,  -0.1502,  -0.2978,  -0.2596,  -0.1093,  -0.2978]
[  0.1009,   0.1264,  -0.2925,  -0.1168,   0.1374,  -0.2962,   0.2241,  -0.1494,  -0.2962,  -0.2646,  -0.1089,  -0.2925]
[   0.104,   0.1262,   -0.282,  -0.1259,   0.1384,   -0.295,    0.215,  -0.1484,   -0.295,  -0.2621,  -0.1091,   -0.282]
[  0.1136,   0.1254,   -0.269,   -0.135,   0.1395,   -0.294,   0.2059,  -0.1473,   -0.294,  -0.2534,  -0.1098,   -0.269]
[  0.1281,    0.124,  -0.2563,  -0.1438,   0.1405,  -0.2933,   0.1971,  -0.1463,  -0.2933,  -0.2401,   -0.111,  -0.2563]
[  0.1459,   0.1223,  -0.2466,  -0.1523,   0.1415,  -0.2929,   0.1886,  -0.1453,  -0.2929,  -0.2235,  -0.1126,  -0.2466]
[  0.1656,   0.1203,  -0.2426,  -0.1607,   0.1424,  -0.2926,   0.1802,  -0.1444,  -0.2926,  -0.2053,  -0.1145,  -0.2426]
[  0.1936,   0.1174,  -0.2463,  -0.1607,   0.1424,  -0.2926,   0.1802,  -0.1444,  -0.2926,  -0.1787,  -0.1172,  -0.2463]"""

grf_positions_b = np.array(grf_positions_b.replace("[", "").replace(
    "]", "").replace(",", "\n").split("\n"),
                           dtype=float)
grf_positions_b = grf_positions_b.reshape((-1, 12))
grf_positions_w = """[    0.4774,     0.1762,    0.05043,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,      0.105,   -0.05615,    0.04998]
[    0.4841,     0.1891,    0.04671,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1133,     -0.044,     0.0463]
[    0.4908,     0.2021,    0.03737,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1218,   -0.03171,    0.03704]
[    0.4968,     0.2135,    0.02522,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1292,   -0.02087,      0.025]
[    0.5015,     0.2227,    0.01308,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1352,   -0.01223,    0.01296]
[    0.5046,     0.2287,   0.003737,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1391,  -0.006508,   0.003704]
[    0.5058,     0.2309,          0,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.1542,     0.2836,    0.00366,     0.4672,    0.02665,   0.003579,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.1619,     0.2813,    0.01293,     0.4765,    0.02265,    0.01286,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.1735,     0.2777,    0.02498,     0.4904,    0.01661,    0.02493,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,      0.188,     0.2733,    0.03702,     0.5079,   0.009042,      0.037,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2045,     0.2683,    0.04629,     0.5277,  0.0004514,    0.04629,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,      0.222,     0.2629,       0.05,     0.5487,   -0.00865,       0.05,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2395,     0.2576,     0.0463,     0.5697,   -0.01775,     0.0463,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,      0.256,     0.2526,    0.03704,     0.5895,   -0.02634,    0.03704,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2705,     0.2481,      0.025,      0.607,   -0.03391,      0.025,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2821,     0.2446,    0.01296,     0.6209,   -0.03995,    0.01296,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2898,     0.2423,   0.003704,     0.6301,   -0.04395,   0.003704,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.1406,  -0.004438,          0]
[    0.5102,     0.2305,   0.003704,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.1448,  -0.004871,   0.003704]
[    0.5225,     0.2292,    0.01296,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.1565,  -0.006066,    0.01296]
[    0.5411,     0.2273,      0.025,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.1741,  -0.007872,      0.025]
[    0.5644,     0.2249,    0.03704,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.1963,   -0.01014,    0.03704]
[    0.5908,     0.2222,     0.0463,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.2214,   -0.01271,     0.0463]
[    0.6188,     0.2193,       0.05,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,      0.248,   -0.01543,       0.05]
[    0.6468,     0.2165,     0.0463,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.2746,   -0.01815,     0.0463]"""

grf_positions_w = np.array(grf_positions_w.replace("[", "").replace(
    "]", "").replace(",", "\n").split("\n"),
                           dtype=float)
grf_positions_w = grf_positions_w.reshape((-1, 12))
grf_plan = """[     -0,      -0,       0,  -44.99,  -44.99,     150,   -2.99,  -2.999,   10.01,      -0,      -0,       0]
[     -0,      -0,       0,  -18.13,  -34.12,   113.7,  -3.217,  -3.226,   10.76,      -0,      -0,       0]
[     -0,      -0,       0,   30.59,  -30.64,   102.1,  -9.669,  -9.686,   32.29,      -0,      -0,       0]
[     -0,      -0,       0,   33.33,  -33.35,   111.2,  -12.03,  -12.09,   40.31,      -0,      -0,       0]
[     -0,      -0,       0,    36.4,  -36.41,   121.4,   9.111,  -14.55,   48.52,      -0,      -0,       0]
[     -0,      -0,       0,   42.32,  -42.33,   141.1,   13.42,  -13.47,   44.91,      -0,      -0,       0]
[  3.001, -0.9456,   10.06,      -0,      -0,       0,      -0,      -0,       0,   6.534,  -6.502,   21.82]
[  5.736,    5.59,   19.17,      -0,      -0,       0,      -0,      -0,       0,   7.092,  -6.989,   23.68]
[  9.592,   7.586,   32.04,      -0,      -0,       0,      -0,      -0,       0,   10.85,  -10.46,   36.23]
[  12.38,   5.149,   41.35,      -0,      -0,       0,      -0,      -0,       0,    14.9,  -6.565,   49.75]
[  14.27,    2.92,   47.72,      -0,      -0,       0,      -0,      -0,       0,   18.84,  -1.544,   62.94]
[  16.11,   1.567,   54.05,      -0,      -0,       0,      -0,      -0,       0,    21.8,   1.348,   73.15]
[  18.32,  0.3076,   63.01,      -0,      -0,       0,      -0,      -0,       0,   19.85,   2.026,   79.71]
[  17.21,  -1.984,   75.18,      -0,      -0,       0,      -0,      -0,       0,   15.67,  0.5792,   81.42]
[  15.69,  -4.614,   85.17,      -0,      -0,       0,      -0,      -0,       0,   13.72,  -1.562,   79.38]
[  14.62,   -7.06,   91.96,      -0,      -0,       0,      -0,      -0,       0,   12.34,  -3.587,   75.25]
[  13.81,  -9.135,   95.82,      -0,      -0,       0,      -0,      -0,       0,   11.33,  -5.376,   69.98]
[  12.64,  -10.34,   97.05,      -0,      -0,       0,      -0,      -0,       0,   10.48,  -7.074,   64.63]
[     -0,      -0,       0,   1.644,  -10.16,   99.29,    1.77,  -9.783,   51.36,      -0,      -0,       0]
[     -0,      -0,       0,  -3.151,  -5.516,   90.38,  -2.531,  -4.767,   63.85,      -0,      -0,       0]
[     -0,      -0,       0,   -5.77,  -1.458,   81.79,  -5.499,  -1.146,   72.36,      -0,      -0,       0]
[     -0,      -0,       0,  -6.967,   1.512,   74.43,  -7.066,   1.401,   77.49,      -0,      -0,       0]
[     -0,      -0,       0,  -6.967,   3.241,   68.91,  -7.221,   2.968,   79.74,      -0,      -0,       0]
[     -0,      -0,       0,   -5.77,   3.695,   65.54,  -5.987,   3.471,   79.19,      -0,      -0,       0]
[     -0,      -0,       0,  -4.517,   2.613,   65.28,   -4.66,   2.462,   74.71,      -0,      -0,       0]"""
grf_plan = np.array(grf_plan.replace("[",
                                     "").replace("]",
                                                 "").replace(",",
                                                             "\n").split("\n"),
                    dtype=float)
grf_plan = grf_plan.reshape((-1, 12))
body_plan = """[    0.3021,    0.08888,     0.3317,     0.3966,    -0.3794,   -0.08081,     0.1943,     0.3215,   -0.04255,      -11.8,     -1.376,     0.4041]
[    0.3049,    0.09338,     0.3311,     0.2323,    -0.3993,   -0.08321,     0.1454,     0.2726,   -0.01687,     -5.882,     0.4581,   -0.00849]
[     0.307,    0.09747,     0.3309,     0.1435,    -0.3926,   -0.08162,     0.1221,     0.2318,    -0.0281,     -1.849,      1.715,  -0.005792]
[    0.3089,     0.1009,     0.3304,     0.1143,    -0.3671,   -0.07774,     0.1449,     0.1878,    -0.0285,     -1.201,      1.922,    -0.1764]
[     0.311,     0.1038,       0.33,    0.09599,    -0.3382,   -0.07703,     0.1682,     0.1382,   -0.01026,    -0.9989,       2.16,    -0.3548]
[    0.3136,     0.1058,     0.3299,    0.08178,    -0.3054,   -0.07935,     0.2179,    0.08257,    0.02808,     -0.677,      2.046,    -0.4304]
[    0.3168,     0.1071,     0.3303,    0.07286,    -0.2743,   -0.08347,     0.2787,    0.02165,    0.08401,      0.612,      1.998,    -0.5177]
[     0.321,     0.1074,     0.3315,     0.0836,    -0.2439,   -0.08925,     0.2892,    0.01352,   -0.02834,     -0.752,      1.937,    -0.4205]
[    0.3253,     0.1076,     0.3311,    0.07328,    -0.2144,   -0.09323,     0.3032,    0.01199,    -0.1287,     -0.656,      1.787,    -0.2762]
[    0.3299,     0.1078,     0.3292,    0.06392,    -0.1873,   -0.09545,     0.3255,   0.008852,    -0.2013,    -0.5252,      1.533,   -0.08485]
[    0.3348,     0.1079,     0.3262,      0.056,    -0.1643,   -0.09524,     0.3553,   0.007307,     -0.249,    -0.4224,      1.239,    0.04976]
[    0.3401,      0.108,     0.3224,    0.04937,    -0.1458,   -0.09344,     0.3914,   0.008809,    -0.2754,    -0.3388,     0.9645,     0.1204]
[     0.346,     0.1082,     0.3183,    0.04392,    -0.1314,   -0.09089,     0.4328,    0.01199,    -0.2836,    -0.2714,      0.715,     0.1507]
[    0.3525,     0.1083,     0.3141,    0.03949,    -0.1208,   -0.08814,     0.4744,    0.01454,     -0.275,    -0.2183,     0.5204,     0.1358]
[    0.3596,     0.1086,     0.3099,    0.03593,    -0.1131,   -0.08578,     0.5103,    0.01301,    -0.2512,    -0.1751,     0.4056,     0.1007]
[    0.3672,     0.1088,     0.3062,     0.0331,    -0.1071,   -0.08404,     0.5424,   0.006263,    -0.2187,    -0.1396,     0.3231,    0.07075]
[    0.3754,     0.1088,     0.3029,    0.03088,    -0.1023,   -0.08281,     0.5719,  -0.005361,    -0.1833,    -0.1103,     0.2587,    0.05035]
[     0.384,     0.1088,     0.3001,    0.02913,   -0.09841,   -0.08193,     0.5993,    -0.0212,    -0.1494,   -0.08618,     0.2089,    0.04294]
[    0.3929,     0.1084,     0.2979,    0.02777,    -0.0953,   -0.08119,     0.6246,   -0.04022,    -0.1201,   -0.06633,      0.191,    0.05759]
[    0.4023,     0.1078,     0.2961,    0.02668,   -0.09246,   -0.08024,     0.6283,   -0.06199,    -0.1027,    -0.0488,      0.142,    0.01254]
[    0.4117,     0.1069,     0.2945,    0.02593,   -0.09033,      -0.08,     0.6221,   -0.07322,   -0.08152,   -0.03604,     0.1074,  0.0004511]
[    0.4211,     0.1058,     0.2933,    0.02538,   -0.08872,   -0.07995,     0.6098,   -0.07606,   -0.06039,   -0.02566,    0.07889,  -0.001049]
[    0.4302,     0.1047,     0.2924,      0.025,   -0.08754,   -0.07993,     0.5945,   -0.07288,   -0.04169,   -0.01721,    0.05508, -0.0003251]
[    0.4391,     0.1036,     0.2918,    0.02474,   -0.08671,   -0.07992,      0.579,    -0.0661,   -0.02655,   -0.01032,    0.03524,  0.0002021]
[    0.4478,     0.1026,     0.2914,    0.02458,   -0.08619,    -0.0799,     0.5661,   -0.05828,    -0.0157,  -0.004712,    0.01997,  -5.49e-05]
[    0.4563,     0.1017,     0.2912,    0.02451,   -0.08589,    -0.0799,     0.5561,   -0.05274,   -0.01002, -3.713e-05,   0.005272, -6.666e-05]"""
body_plan = np.array(body_plan.replace("[", "").replace("]", "").replace(
    ",", "\n").split("\n"),
                     dtype=float)
body_plan = body_plan.reshape((-1, 12))

contact_plan = """0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0"""
contact_plan = contact_plan.replace("\n", "").split(" ")
contact_plan = np.array(contact_plan, dtype=int).reshape((-1, 4))

ref_body_state = """[  0.3021,  0.08888,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3091,  0.08832,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3166,  0.08771,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3241,  0.08711,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3315,   0.0865,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[   0.339,   0.0859,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3465,  0.08529,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[   0.354,  0.08468,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3614,  0.08408,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3689,  0.08347,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3764,  0.08287,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3839,  0.08226,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3913,  0.08166,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3988,  0.08105,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4063,  0.08045,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4138,  0.07984,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4212,  0.07924,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4287,  0.07863,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4362,  0.07803,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4437,  0.07742,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4511,  0.07681,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4586,  0.07621,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4661,   0.0756,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4736,    0.075,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[   0.481,  0.07439,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4885,  0.07379,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]"""
ref_body_state = np.array(ref_body_state.replace("[",
                                                 "").replace("]", "").replace(
                                                     ",", "\n").split("\n"),
                          dtype=float)
ref_body_state = ref_body_state.reshape((-1, 12))

next_state = """[  0.3057]
[ 0.09647]
[  0.3305]
[  0.1079]
[ -0.4087]
[-0.08604]
[  0.1049]
[  0.3057]
[-0.05265]
[  -13.78]
[ -0.8608]
[ -0.1516]"""
next_state = np.array(next_state.replace("[", "").replace("]", "").replace(
    ",", "\n").split("\n"),
                      dtype=float)
next_state = next_state.reshape((-1, 12))


### Test

In [ ]:
x0_real = body_plan[0, :]
x_ref_real = ref_body_state[0:N, :].reshape((-1))
x_foots_real = grf_positions_w[0:N, :].reshape(-1)
contacts_real = contact_plan[0:N, :]
mpc_res = mpc(x0_real, x_ref_real, x_foots_real, contacts_real)
%timeit -r 40 mpc(x0_real, x_ref_real, x_foots_real, contacts_real)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   6.19ms (386.88us) 426.75us ( 26.67us)        16
       nlp_g  |   6.80ms (424.88us) 468.37us ( 29.27us)        16
  nlp_grad_f  |  11.04ms (689.75us) 854.85us ( 53.43us)        16
  nlp_hess_l  | 342.31ms ( 24.45ms)  23.72ms (  1.69ms)        14
   nlp_jac_g  |  65.95ms (  4.12ms)   4.81ms (300.75us)        16
       total  | 858.15ms (858.15ms)  69.61ms ( 69.61ms)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  12.78ms (399.41us) 846.60us ( 26.46us)        32
       nlp_g  |  13

In [ ]:
1.10 / 0.078


14.102564102564104

In [ ]:
u_res = mpc_res[0].full().reshape((-1, 12))
u_res

array([[ 6.40287082e-20,  3.16566198e-18, -8.82347358e-09,
        -1.83707428e+01, -2.95929343e+00,  6.93353999e+01,
        -8.03433762e+00,  9.64438988e+00,  5.26538794e+01,
         5.97281836e-19,  2.30670947e-18, -8.82347358e-09],
       [-3.40343388e-21,  2.25410665e-18, -8.82347358e-09,
        -4.58719063e+00, -1.04092763e+01,  6.24612809e+01,
        -4.74092762e+00, -1.05967948e+01,  6.28843852e+01,
         4.53004145e-19,  1.52501190e-18, -8.82347358e-09],
       [ 9.89630551e-21,  1.38822859e-18, -8.82347358e-09,
        -4.06848811e-01, -8.08843576e+00,  6.38270107e+01,
        -1.28613237e+00, -9.16134383e+00,  6.74631260e+01,
         3.20070737e-19,  8.97058430e-19, -8.82347358e-09],
       [ 1.50634373e-20,  6.27368208e-19, -8.82347358e-09,
         1.06701310e+00, -5.86369631e+00,  6.47718364e+01,
        -3.58937976e-01, -7.60316303e+00,  6.92329596e+01,
         1.76311850e-19,  3.73934154e-19, -8.82347358e-09]])

In [ ]:
x_res = mpc_res[1].full().reshape((-1, 3))
x_res

array([[ 3.02100000e-01,  8.88800000e-02,  3.31700000e-01],
       [ 3.96600000e-01, -3.79400000e-01, -8.08100000e-02],
       [ 1.94300000e-01,  3.21500000e-01, -4.25500000e-02],
       [-1.18000000e+01, -1.37600000e+00,  4.04100000e-01],
       [ 3.07929000e-01,  9.85250000e-02,  3.30423500e-01],
       [ 4.44994147e-02, -4.22158760e-01, -8.59385190e-02],
       [ 1.36646986e-01,  3.36096280e-01, -7.04978618e-02],
       [-1.87860215e-01, -3.84477117e-01, -3.85709391e-01],
       [ 3.12028410e-01,  1.08607888e-01,  3.28308564e-01],
       [ 4.42863367e-02, -4.33166910e-01, -9.91733808e-02],
       [ 1.16279916e-01,  2.90231496e-01, -9.11173682e-02],
       [-2.12909990e-02, -6.38029608e-02, -3.56707674e-01],
       [ 3.15516807e-01,  1.17314833e-01,  3.25575043e-01],
       [ 4.86308304e-02, -4.34605359e-01, -1.11045341e-01],
       [ 1.12583450e-01,  2.52568222e-01, -9.87576810e-02],
       [-7.97023299e-03, -5.03000429e-03, -3.72837747e-01],
       [ 3.18894311e-01,  1.24891880e-01

In [ ]:
mpc.generate("mpc2_casadi_dynamics.c")
C = Importer('mpc2_casadi_dynamics.c', 'shell')

CasADi - 2023-05-05 20:52:19 WARNING("The function "solver", which is of type "IpoptInterface" cannot be code generated. The generation will proceed, but compilation of the code will not be possible.") [.../casadi/core/function_internal.cpp:2642]
mpc2_casadi_dynamics.c: In function ‘mpc2_casadi_dynamics_f1’:
mpc2_casadi_dynamics.c:94:4: error: #error Code generation not supported for IpoptInterface
   94 |   #error Code generation not supported for IpoptInterface
      |    ^~~~~
CasADi - 2023-05-05 20:52:19 WARNING("Failed to remove ./tmp_casadi_compiler_shellAOPO30.so") [.../casadi/solvers/shell_compiler.cpp:66]


RuntimeError: .../casadi/solvers/shell_compiler.cpp:231: Compilation failed. Tried "gcc -fPIC -c mpc2_casadi_dynamics.c -o ./tmp_casadi_compiler_shellAOPO30.o"